In [ ]:
import requests
import re

file_path = r"C:\Users\Jofu2\Desktop\rosalind_mprt_4.txt"
results = {}

#########################################################################

#OPENING THE FILE 

with open(file_path, 'r') as file:
    for line in file:
        uniprotID = line.strip()  
        protein_sequence = sequence_extraction(uniprotID)
        if protein_sequence:
            glycosylation_sites = n_glycosylation_search(protein_sequence) #searches and saves the locations
            results[uniprotID] = glycosylation_sites 

#############################################################################      

#TAKING THE FILE AND THE ID AND SEARCHING THE UNIPROT DATABASE FOR IT
            
def sequence_extraction(uniprotID):

    proteinurl = "http://uniprot.org/uniprot/" + uniprotID + ".txt"
    file_contents = requests.get(proteinurl)

    if file_contents.status_code == 200: #http code to say that the webpage is ok 
        lines = file_contents.text.split('\n')
        sequence = "" #variable sequence is empty but I can append data to it 
        in_sequence = False

        for line in lines: #ok this is my for loop to search the .txt file for "SQ" going through each line of the text file
            if line.startswith("SQ"):
                in_sequence = True
                continue  #continue used in loops to skip the rest of the code and move to the next iteration.

            if in_sequence and line.strip():
                line = line.replace(" ", "").replace("/", "") #replaces and any spaces or / with nothing
                sequence += line # Concatenate the sequence lines to the variable sequence 
                
        if sequence:  
            return sequence
        else:
            return "No sequence found for the provided UniProt ID."
    else:
        return "UNIPROT ERROR: Invalid UniProt ID or resource not found."

######################################################################

#SEARCH FOR THE N GLYCOSYLATION MOTIF
     
def n_glycosylation_search(protein_sequence):

    n_glycosylation = (re.finditer(r"N(?=([^P][ST][^P]))", protein_sequence))  #Remove findall, finditer does exactly the same more efficiently 
    positions = [match.start() + 1 for match in n_glycosylation] #+1 to convert form counting from 0 to counting from 1. Instead of f string literal 
    return positions 

######################################################################

for uniprotID, sites in results.items():
    print(f"UniProt ID: {uniprotID}, N-glycosylation site: {sites}")